In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 模型参数
INPUT_DIM = 11
HIDDEN_DIM = 64
N_LAYERS = 2

In [ ]:
# 注意：这里的模型定义需要和训练时的代码保持完全一致

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return hidden, cell
        
# --- 核心：特征提取器类 ---

class FeatureExtractor:
    def __init__(self, model_path, input_dim, hidden_dim, n_layers, dropout=0.0):
        """
        初始化特征提取器。

        参数:
            model_path (str): 训练好的完整Seq2Seq模型权重文件路径 (例如 'feature_extractor_model.pt')
            input_dim (int): 输入特征维度 (11)
            hidden_dim (int): LSTM隐藏层维度，也即输出特征的维度 (例如 64)
            n_layers (int): LSTM层数 (例如 2)
            dropout (float): Dropout比例，加载时应与训练时一致
        """
        # 检查设备
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"FeatureExtractor is using device: {self.device}")

        # 1. 实例化Encoder
        # 我们只需要加载Encoder部分即可
        self.encoder = Encoder(input_dim, hidden_dim, n_layers, dropout).to(self.device)

        # 2. 加载训练好的权重
        # 注意：我们加载的是包含Encoder和Decoder的完整模型权重，
        # 但PyTorch会自动匹配名称，只加载Encoder对应的权重。
        # 为了更干净，可以只保存和加载encoder的state_dict，但这样也能工作。
        
        # 创建一个临时的完整模型来加载字典
        # (这是一个小技巧，以避免处理state_dict中不匹配的键)
        class DummyDecoder(nn.Module):
            def __init__(self): super().__init__(); self.dummy = nn.Linear(1,1)
        
        class DummySeq2Seq(nn.Module):
            def __init__(self, encoder):
                super().__init__()
                self.encoder = encoder
                self.decoder = DummyDecoder() # 只是占位
        
        temp_full_model = DummySeq2Seq(self.encoder)
        temp_full_model.load_state_dict(torch.load(model_path, map_location=self.device))
        
        print(f"Successfully loaded encoder weights from {model_path}")

        # 3. 设置为评估模式
        # 这是非常重要的一步！它会关闭Dropout等训练特有的层。
        self.encoder.eval()

    def extract_feature(self, sequence_data):
        """
        从一个4秒(200个点)的序列中提取特征向量。

        参数:
            sequence_data (np.ndarray): 输入的传感器数据，形状必须为 (200, 11)

        返回:
            np.ndarray: 提取出的特征向量，形状为 (hidden_dim,)
        """
        # --- 输入验证 ---
        if not isinstance(sequence_data, np.ndarray) or sequence_data.shape != (200, 11):
            raise ValueError("Input data must be a numpy array of shape (200, 11)")

        # --- 特征提取核心逻辑 ---
        with torch.no_grad(): # 关闭梯度计算，加速推理
            # 1. 将Numpy数组转换为PyTorch张量
            input_tensor = torch.tensor(sequence_data, dtype=torch.float32).to(self.device)

            # 2. 增加Batch维度
            # 模型的LSTM层期望的输入是 (batch_size, seq_len, input_dim)
            # 所以 (200, 11) 需要变成 (1, 200, 11)
            input_tensor = input_tensor.unsqueeze(0)

            # 3. 通过Encoder进行前向传播
            hidden_state, _ = self.encoder(input_tensor)
            # hidden_state 的形状是 (n_layers, batch_size, hidden_dim)

            # 4. 提取我们需要的特征向量
            # 通常我们使用最后一层的隐藏状态作为特征
            feature_vector_tensor = hidden_state[-1, :, :] # 取最后一层, shape: (1, hidden_dim)

            # 5. 去掉Batch维度，并转换回Numpy数组
            feature_vector_tensor = feature_vector_tensor.squeeze(0) # Shape: (hidden_dim)
            feature_vector_np = feature_vector_tensor.cpu().numpy()

            return feature_vector_np

In [ ]:
# --- 使用流程 ---
print("\n--- Initializing Feature Extractor ---")
# 2. 定义模型参数 (必须和训练时完全一致)
MODEL_PARAMS = {
    'model_path': 'feature_extractor_model.pt',
    'input_dim': INPUT_DIM,
    'hidden_dim': HIDDEN_DIM,
    'n_layers': N_LAYERS,
    'dropout': 0.1
}

# 3. 实例化提取器
# 在您的边缘节点程序启动时，只需要执行一次这个实例化操作
extractor = FeatureExtractor(**MODEL_PARAMS)

print("\n--- Performing Feature Extraction ---")
# 4. 模拟从传感器获得的一个4秒（200个点）的数据
# 在实际应用中，这将是您实时收集的数据
new_sensor_data = np.random.rand(200, 11).astype(np.float32)
print(f"Input data shape: {new_sensor_data.shape}")

# 5. 调用方法提取特征
feature = extractor.extract_feature(new_sensor_data)

# 6. 查看结果
print(f"Successfully extracted feature vector.")
print(f"Feature vector shape: {feature.shape}")
print("This feature vector can now be sent to the main server.")
print("Feature vector sample:", feature[:5]) # 查看具体数值